# 与えられたデータに外れ値が頻出する件について
## 要件
たとえば 2012-07-12では

```
VZ      446200  448200  442700  446700
GE      194800  196200  193700  194400
GM      196100  196100  192400  193300
GS      942900  947000  932500  940200 ...
```

のようになっているが、2014-12-31では
```
VZ      47.24   47.4218 46.71   46.85
GE      25.65   25.65   25.27   25.34
GM      35.49   35.49   34.83   34.83
GS      196.32  196.83  193.83  193.86
PFE     31.38   31.69   31.1    31.2
CAT     92.74   92.95   91.41   91.54 ...
```
となって明らかにおかしい。この傾向は全体で見られるので、集計方法や桁数が変更された可能性がある。
具体的な統計をとって、どの期間からどの期間でこのような形式がみられるのかを調べたい。

調査しました。
 - 20131202以前と以後でわかれる
 - 20131202以降は問題なし
 - 20131202以前では、データは不揃い  
 
VZ社
![download](https://user-images.githubusercontent.com/27924055/30359547-8916ab84-9886-11e7-9bff-0586253b652c.png)
 境目になっているのは 20131202です。
 - また、20131202以前でも、データが（それ以降と）整合的な会社もあります。
 CSCO.O社
![download-1](https://user-images.githubusercontent.com/27924055/30359616-f106f6ea-9886-11e7-9a4f-efd5510fad4e.png)

## やること

 - 前日比に単位を統一したときに何が起こるか
     - ある区間で外れ値となるので、区間の始まりと終わりのみ除外すれば良い
         - たとえば、VZ社では2012-01-03 ~ 2013-12-03の2年間が外れ値となっているため、その2日のみを除外する
         - 他の会社で、その区間は同一か？
             - 結論から言うと同一でない
             - つまり、外れ値になっている区間を見つけ出して、その日を除いて返す関数が必要

In [42]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append("../../lib/")
from load_tsvs import load_tsvs


In [308]:
# TSVデータの読み出し
targetDir = "../stockValueFull/"
df = load_tsvs(dir_path=targetDir)
df=df.set_index("date")

In [91]:
# df_vzに前日比を追加
df_vz = df[df["COMPANY"] == vz]
pct = df_vz[['Open', 'High', 'Low', 'Close']].pct_change()
pct.columns = ['Open_p', 'High_p', 'Low_p', 'Close_p']
df_vz = pd.concat([df_vz,pct],axis=1)

# dateをインデックスに
df_vz = df_vz.set_index("date")

In [303]:
def removeOutlier(df,corp):
    """
    すでにdfは前日比に修正済みのものとする
    """
    df_corp = df[df["COMPANY"] == corp]
    pct = df_corp[['Open', 'High', 'Low', 'Close']].pct_change()
    pct.columns = ['Open_p', 'High_p', 'Low_p', 'Close_p']
    df_corp = pd.concat([df_corp,pct],axis=1)
    
    for i in range(6):
        oStart = df_corp[df_corp["Close_p"] == df_corp.Close_p.max()].index
        oEnd = df_corp[df_corp["Close_p"] == df_corp.Close_p.min()].index
        df_corp = df_corp.drop(oStart)
        df_corp = df_corp.drop(oEnd)
    
    return df_corp


In [309]:
resultDf = removeOutlier(df,"VZ")
for comp in df.COMPANY.unique():
    if comp == "VZ":
        pass
    else:
        _df = removeOutlier(df,comp)
        resultDf = pd.concat([resultDf,_df])

,COMPANY,Open,High,Low,Close,Open_p,High_p,Low_p,Close_p
date,,,,,,,,,
2005-01-03,VZ,40.510,41.040,40.370,40.485,nan,nan,nan,nan
2005-01-04,VZ,40.730,40.730,39.890,40.136,0.005,-0.008,-0.012,-0.009
2005-01-05,VZ,40.140,40.510,39.970,40.020,-0.014,-0.005,0.002,-0.003
2005-01-06,VZ,40.000,40.000,39.150,39.560,-0.003,-0.013,-0.021,-0.011
2005-01-07,VZ,39.600,39.900,39.290,39.300,-0.010,-0.003,0.004,-0.007
2005-01-10,VZ,39.460,39.460,38.830,39.030,-0.004,-0.011,-0.012,-0.007
2005-01-11,VZ,39.040,39.040,38.580,38.770,-0.011,-0.011,-0.006,-0.007
2005-01-12,VZ,38.500,38.520,37.800,38.230,-0.014,-0.013,-0.020,-0.014
2005-01-13,VZ,38.000,38.010,36.880,37.070,-0.013,-0.013,-0.024,-0.030


In [311]:
resultDf = resultDf.sort_index()

In [313]:
resultDf = resultDf.dropna(how='any')

In [314]:
resultDf.to_csv("resultDf.csv")

In [332]:
dateList = list(resultDf.index.unique())

In [334]:
for d in dateList:
    aDay = resultDf[resultDf.index == d]
    aDay.to_csv("../processed/"+ str(d) + ".tsv",sep='\t')

In [328]:
resultDf[resultDf.index == "2005-01-04"]

,COMPANY,Open,High,Low,Close,Open_p,High_p,Low_p,Close_p
date,,,,,,,,,
2005-01-04,PFE,26.450,26.600,26.050,26.400,-0.024,-0.027,-0.014,-0.004
2005-01-04,MSFT.O,26.840,27.090,26.660,26.800,0.002,0.006,0.000,0.002
2005-01-04,NKE,35.260,35.500,34.360,34.670,-0.009,-0.013,-0.023,-0.021
2005-01-04,JPM,39.000,39.040,38.330,38.410,-0.004,-0.015,-0.018,-0.019
2005-01-04,HD,43.240,43.260,42.520,42.520,0.007,0.001,-0.010,-0.013
2005-01-04,BAC,45.330,46.730,45.330,45.900,-0.033,-0.004,-0.019,-0.012
2005-01-04,V,31.740,32.310,31.740,32.162,0.003,0.002,0.004,0.011
2005-01-04,HON,41.450,41.610,40.920,40.960,-0.010,-0.008,-0.012,-0.016
2005-01-04,KO,32.770,32.800,32.270,32.270,0.014,0.002,0.001,-0.007
